# Setup

In [ ]:
!pip uninstall torch_geometric -y
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install torch-spline-conv
!pip install torch-geometric
!pip install dgl
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install torch_geometric
!pip install torch-sparse
!pip install pyg_lib
import pyglib
import torch_sparse
import torch_geometric

# Preprocessing


In [ ]:
import pandas as pd
# df1=pd.read_csv('SA02601_v6.csv')
plant_feats=pd.read_csv('SA02602_v5.csv')[['PLOT_ID','PLTSP_CODE','NO_STALK','NO_FLWS']].iloc[1:]
int_feats=pd.read_csv('SA02601_v6.csv')[['PLOT_ID','PLTSP_CODE','VISSP_CODE','VISSP_TYPE','VISSP_NO','NO_INT']]
int_feats['PLTSP_CODE']=int_feats['PLTSP_CODE'].fillna("").astype('string')
int_feats=int_feats[int_feats['PLTSP_CODE']!='ALLIAMPL']

plant_feats['PLTSP_CODE']=plant_feats['PLTSP_CODE'].fillna("").astype('string')
plant_feats=plant_feats[plant_feats['PLTSP_CODE']!='']
plant_feats=plant_feats[~plant_feats['NO_FLWS'].isna()]
plant_feats=plant_feats[~plant_feats['NO_STALK'].isna()]
# plant_feats=plant_feats[plant_feats['PLTSP_CODE']!='ALLIAMPL']

int_feats=int_feats[int_feats['PLTSP_CODE']!='']
int_feats=int_feats[int_feats['PLTSP_CODE'].isin(list(plant_feats['PLTSP_CODE']))]
# int_feats=int_feats[int_feats['PLTSP_CODE']!='']
# data=data.rename({'PLTSP_NAME_x':'PLTSP_NAME'})

In [ ]:
visit_feats=int_feats[['VISSP_CODE','VISSP_TYPE']]
int_feats=int_feats[['PLOT_ID','PLTSP_CODE','VISSP_CODE','VISSP_NO','NO_INT']]
plant_feats=plant_feats.groupby('PLTSP_CODE')[['NO_STALK','NO_FLWS']].mean()
plant_feats['PLTSP_CODE']=plant_feats.index

In [ ]:
from sklearn.preprocessing import LabelEncoder
# CODE=LabelEncoder().fit(codes)
PLTSP_CODE=LabelEncoder().fit(plant_feats['PLTSP_CODE'])
VISSP_CODE=LabelEncoder().fit(visit_feats['VISSP_CODE'])
VISSP_TYPE=LabelEncoder().fit(visit_feats['VISSP_TYPE'])
PLOT_ID=LabelEncoder().fit(int_feats['PLOT_ID'])

In [ ]:
plant_feats['PLTSP_CODE']=PLTSP_CODE.transform(plant_feats['PLTSP_CODE'])

int_feats['PLTSP_CODE']=PLTSP_CODE.transform(int_feats['PLTSP_CODE'])
int_feats['VISSP_CODE']=VISSP_CODE.transform(int_feats['VISSP_CODE'])
int_feats['PLOT_ID']=PLOT_ID.transform(int_feats['PLOT_ID'])

visit_feats['VISSP_CODE']=VISSP_CODE.transform(visit_feats['VISSP_CODE'])
visit_feats['VISSP_TYPE']=VISSP_TYPE.transform(visit_feats['VISSP_TYPE'])

In [ ]:
from sklearn.mixture import GaussianMixture
temp=int_feats.groupby(['PLOT_ID','PLTSP_CODE','VISSP_CODE'])[['VISSP_NO','NO_INT']].mean()
from scipy.special import comb
def binom_pmf(N, k, p):
    # Using gamma function to handle non-integer combinations in binomial coefficient
    return comb(N, k) * (p**k) * ((1 - p)**(N - k))
out=[]
for N,k in zip(temp['VISSP_NO'],temp['NO_INT']):
  if(N<=k):
    out.append(binom_pmf(N,k,0.5))
  else:
    out.append(1)
temp['EDGE_PROB']=out
temp=temp[temp['EDGE_PROB']>0]
# int_feats=temp[['PLOT_ID','PLTSP_CODE','EDGE_PROB']]
temp['PLOT_ID']=[i for i,j,k in temp.index]
temp['PLTSP_CODE']=[j for i,j,k in temp.index]
temp['VISSP_CODE']=[k for i,j,k in temp.index]
temp.index=range(1,len(temp.index)+1)
temp=temp.drop(['NO_INT','VISSP_NO'],axis=1)

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
gmm = GaussianMixture(n_components=3, random_state=42)
interaction_data = np.array(temp['EDGE_PROB']).reshape(-1, 1)
gmm.fit(interaction_data)

def all_plot_prob(interaction_data):
#   print(interaction_data)
  responsibilities = gmm.predict_proba(np.array(interaction_data).reshape(-1,1))

  reduced_edges = np.dot(responsibilities.T, np.array(interaction_data).reshape(-1,1)) / responsibilities.sum(axis=0)
  reduced_edges[reduced_edges==np.inf]=0
  reduced_edges=np.nan_to_num(reduced_edges)
  return reduced_edges.T[0]
# print(temp.groupby(['PLTSP_CODE','VISSP_CODE'])['EDGE_PROB'].mean())
# edge_wt=pd.DataFrame(temp.groupby(['PLTSP_CODE','VISSP_CODE'])['EDGE_PROB'].mean())#.apply(all_plot_prob))
edge_wt=pd.DataFrame(temp.groupby(['PLTSP_CODE','VISSP_CODE'])['EDGE_PROB'].apply(all_plot_prob))
# edge_wt = edge_wt[edge_wt['EDGE_PROB'].apply(lambda x:sum(x)!=0)]
edge_wt['PLTSP_CODE']=[j for j,k in edge_wt.index]
edge_wt['VISSP_CODE']=[k for j,k in edge_wt.index]
edge_wt.index=range(1,len(edge_wt.index)+1)

In [ ]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import torch
import numpy as np
data = HeteroData()# Save node indices:
data["plant"].node_id = torch.arange(len(PLTSP_CODE.classes_)).long()
data["visitor"].node_id = torch.arange(len(VISSP_CODE.classes_)).long()# Add the node features and edge indices:
# vals=train.values.tolist()
# vals.extend(test.values.tolist())
# nfeats_p={i[0]:[i[-1],i[-2]] for i in vals}
# nfeats_i={i[1]:[i[2],i[2]] for i in vals}
temp=[]
for i in range(len(PLTSP_CODE.classes_)):
  data1=plant_feats[plant_feats['PLTSP_CODE']==i]
  temp.append([float(data1['NO_STALK']),float(data1['NO_FLWS'])])
data["plant"].x = torch.Tensor(temp)

temp=[]
for i in range(len(VISSP_CODE.classes_)):
  data1=visit_feats[visit_feats['VISSP_CODE']==i].to_numpy()[0]
  temp.append([data1[1]/len(VISSP_CODE.classes_)])
data["visitor"].x = torch.Tensor(temp).float()

train_edge1=list(set(zip(list(edge_wt['VISSP_CODE']),list(edge_wt['PLTSP_CODE']))))
# train_edge2=[(v,u) for u,v in train_edge1]
# train_edge2.extend(train_edge1)
train_edge=[[u for u,v in train_edge1],[v for u,v in train_edge1]]
data["visitor", "visits", "plant"].edge_index = torch.Tensor(train_edge).long()
data["visitor", "visits", "plant"].edge_attr = torch.Tensor(np.array(list(edge_wt['EDGE_PROB']))).float()# We also need to make sure to add the reverse edges from movies to users
# # in order to let a GNN be able to pass messages in both directions.
# # We can leverage the `T.ToUndirected()` transform for this from PyG:
data = T.ToUndirected()(data)

# GNN


In [ ]:
from torch_geometric.loader import LinkNeighborLoader
transform = T.RandomLinkSplit(
#     num_val=0.1,
#     num_test=0.1,
#     disjoint_train_ratio=0.3,
    neg_sampling_ratio=1.0,
    add_negative_train_samples=True,
    edge_types=("visitor", "visits", "plant"),
    rev_edge_types=("plant", "rev_visits", "visitor"),
)
train_data, val_data, test_data = transform(data)

In [ ]:
edge_label_index = train_data["visitor", "visits", "plant"].edge_label_index
edge_label = train_data["visitor", "visits", "plant"].edge_label
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[5, 2],
#     neg_sampling_ratio=2.0,
    edge_label_index=(("visitor", "visits", "plant"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

In [ ]:
edge_label_index = val_data["visitor", "visits", "plant"].edge_label_index
edge_label = val_data["visitor", "visits", "plant"].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[5, 2],
#     neg_sampling_ratio=2.0,
    edge_label_index=(("visitor", "visits", "plant"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

In [ ]:
edge_label_index = test_data["visitor", "visits", "plant"].edge_label_index
edge_label = test_data["visitor", "visits", "plant"].edge_label
test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=[5, 2],
#     neg_sampling_ratio=2.0,
    edge_label_index=(("visitor", "visits", "plant"), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

In [ ]:
from torch_geometric.nn import SAGEConv, to_hetero
import torch.nn.functional as F
import torch.nn as nn
from torch import Tensor
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        x = self.conv1(x, edge_index)
        x = F.leaky_relu(x)
        x = self.conv2(x, edge_index)
        return x
# Our final classifier applies the dot-product between source and destination
# node embeddings to derive edge-level predictions:
class Classifier(torch.nn.Module):
    def forward(self, x_visitor: Tensor, x_plant: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_visitor = x_visitor[edge_label_index[0]]
        edge_feat_plant = x_plant[edge_label_index[1]]
        # Apply dot-product to get a prediction per supervision edge:
        return (edge_feat_visitor * edge_feat_plant).sum(dim=-1)

class EdgeClassifier(torch.nn.Module):
    def __init__(self, hidden_channels,embedding_size):
        super().__init__()
        self.fc1 = nn.Linear(embedding_size*2, hidden_channels)

        # Non-linearity
        self.sigmoid1 = nn.Sigmoid()

        # Linear function (readout)
#         self.fc2 = nn.Linear(128, 128)
#         self.sigmoid2 = nn.Sigmoid()
        self.fc3 = nn.Linear(hidden_channels, 3)

    def forward(self, x_visitor: Tensor, x_plant: Tensor, edge_label_index: Tensor) -> Tensor:
        # Convert node embeddings to edge-level representations:
        edge_feat_visitor = x_visitor[edge_label_index[0]]
        edge_feat_plant = x_plant[edge_label_index[1]]
#         print(edge_feat_visitor.shape,edge_feat_plant.shape,.shape)
        edge_feats=torch.cat((edge_feat_visitor,edge_feat_plant),1)
#         print(edge_feats.shape)
        x=self.fc1(edge_feats)
        x=self.sigmoid1(x)
#         x=self.fc2(x)
#         x=self.sigmoid2(x)
        x=self.fc3(x)
        return x

class Model(torch.nn.Module):
    def __init__(self, hidden_channels,embedding_size):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        self.plant_lin = torch.nn.Linear(2, embedding_size)
        self.visitor_lin = torch.nn.Linear(1, embedding_size)
        self.visitor_emb = torch.nn.Embedding(data["visitor"].num_nodes, embedding_size)
        self.plant_emb = torch.nn.Embedding(data["plant"].num_nodes, embedding_size)
        # Instantiate homogeneous GNN:
        self.gnn = GNN(embedding_size)
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        self.classifier = Classifier()
        self.edgeclassifier = EdgeClassifier(hidden_channels,embedding_size)
    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "visitor": self.visitor_emb(data["visitor"].node_id)+ self.visitor_lin(data["visitor"].x.float()),
          "plant": self.plant_lin(data["plant"].x) + self.plant_emb(data["plant"].node_id),
        }
        # `x_dict` holds feature matrices of all node types
        # `edge_index_dict` holds all edge indices of all edge types
#         print(x_dict)
        x_dict = self.gnn(x_dict, data.edge_index_dict)
#         print(x_dict)
        pred2 = self.edgeclassifier(
            x_dict["visitor"],
            x_dict["plant"],
            data["visitor", "visits", "plant"].edge_label_index,
        )
        pred = self.classifier(
            x_dict["visitor"],
            x_dict["plant"],
            data["visitor", "visits", "plant"].edge_label_index,
        )
#         print(pred)
        return pred,pred2

model = Model(hidden_channels=64,embedding_size=2)

In [ ]:
import tqdm
import torch.nn.functional as F
from torch.nn import MSELoss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train_loss=[]
val_loss=[]
for epoch in range(1, 100):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):

        optimizer.zero_grad()
        sampled_data.to(device)
        pred,pred2 = model(sampled_data)
#         print(pred2)
#         pred=torch.nan_to_num(pred)
#         print(sampled_data)

        ground_truth_1 = sampled_data["visitor", "visits", "plant"].edge_label.float()
        ground_truth_2 = sampled_data["visitor", "visits", "plant"].edge_attr
#         print(max(sampled_data["visitor", "visits", "plant"].edge_label_index[0]),max(sampled_data["visitor", "visits", "plant"].edge_label_index[1]))
#         print(max(sampled_data["visitor", "visits", "plant"].edge_index[0]),max(sampled_data["visitor", "visits", "plant"].edge_index[1]))
        L1 = F.binary_cross_entropy_with_logits(pred, ground_truth_1)
        temp=[]
#         print(sampled_data["visitor", "visits", "plant"].edge_index,sampled_data["visitor", "visits", "plant"].edge_label_index)

        for u,v in zip(sampled_data["visitor", "visits", "plant"].edge_label_index[0],sampled_data["visitor", "visits", "plant"].edge_label_index[1]):
                flag=True
                for ind,uv in enumerate(zip(sampled_data["visitor", "visits", "plant"].edge_index[0],sampled_data["visitor", "visits", "plant"].edge_index[1])):
                    u1,v1=uv
                    if(u1==u and v1==v):
                        temp.append(ground_truth_2[ind].tolist())
                        flag=False
                        break
                if(flag):
                    temp.append([0.0,0.0,0.0])

#         print(ind1)
        ground_truth_2=torch.Tensor(temp)
#         print(ground_truth_1.shape,ground_truth_2.shape)
#         print(pred.shape,pred2.shape)
        L2 = F.mse_loss(pred2, ground_truth_2)
        loss=L1+L2
        train_loss.append(loss)
#         VAL LOSS


#         loss = MSELoss(pred, ground_truth)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
#         break
    total_loss_val = total_examples_val = 0
    for sampled_data in tqdm.tqdm(val_loader):
        pred,pred2 = model(sampled_data)
        ground_truth_1 = sampled_data["visitor", "visits", "plant"].edge_label.float()
        ground_truth_2 = sampled_data["visitor", "visits", "plant"].edge_attr
        L1 = F.binary_cross_entropy_with_logits(pred, ground_truth_1)
        temp=[]
        for u,v in zip(sampled_data["visitor", "visits", "plant"].edge_label_index[0],sampled_data["visitor", "visits", "plant"].edge_label_index[1]):
                flag=True
                for ind,uv in enumerate(zip(sampled_data["visitor", "visits", "plant"].edge_index[0],sampled_data["visitor", "visits", "plant"].edge_index[1])):
                    u1,v1=uv
                    if(u1==u and v1==v):
                        temp.append(ground_truth_2[ind].tolist())
                        flag=False
                        break
                if(flag):
                    temp.append([0.0,0.0,0.0])
        ground_truth_2=torch.Tensor(temp)
        L2 = F.mse_loss(pred2, ground_truth_2)
        loss=L1+L2
        total_loss_val += float(loss) * pred.numel()
        total_examples_val += pred.numel()
#         VAL LOSS
        val_loss.append(total_loss_val/total_examples_val)
    torch.save(model.state_dict(), '/home/arka/Downloads/GNN/checkpoints/model_'+str(epoch+1)+'.pth')
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}, Val Loss: {total_loss_val/total_examples_val:.4f}")

In [ ]:
# Define the validation seed edges:
# torch.save(model.state_dict(), '/home/arka/Downloads/GNN/model_100_3.pth')
model.load_state_dict(torch.load('/home/arka/Downloads/GNN/checkpoints/model_100_64_2.pth'))
# import matplotlib.pyplot as plt
# train_loss=[i.detach().numpy() for i in train_loss]
# val_loss=[i.detach().numpy() for i in val_loss]
# plt.plot(np.array(train_loss))
# plt.plot(np.array(val_loss))
# plt.show()

<All keys matched successfully>

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
import tqdm
import torch.nn.functional as F
from torch.nn import MSELoss
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")
model = model.to(device)
preds = []
ground_truths = []
print(len(test_loader))
for sampled_data in tqdm.tqdm(test_loader):
    with torch.no_grad():
        sampled_data.to(device)
#         print(sampled_data["visitor",'visits','plant'].edge_label)
        preds.append(model(sampled_data)[0])
        ground_truths.append(sampled_data["visitor",'visits','plant'].edge_label)
pred = torch.cat(preds, dim=0).cpu().numpy()
print(pred.shape)
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
pred=np.nan_to_num(pred)
temp=pred
temp=[0 if i <0 else 1 for i in temp]
pred=temp
print(ground_truth)
cm = confusion_matrix(ground_truth, pred)
ConfusionMatrixDisplay(cm).plot()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC: {auc:.4f}")

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
val_loader=test_loader
preds = []
ground_truths = []
MSE=0
for sampled_data in tqdm.tqdm(test_loader):
    with torch.no_grad():
        sampled_data.to(device)
#         print(sampled_data)
        pred,pred2 = model(sampled_data)
        ground_truth_1 = sampled_data["visitor", "visits", "plant"].edge_label.float()
        ground_truth_2 = sampled_data["visitor", "visits", "plant"].edge_attr
        preds.append(pred)
        ground_truths.append(sampled_data["visitor",'visits','plant'].edge_label)
        temp=[]

        for u,v in zip(sampled_data["visitor", "visits", "plant"].edge_label_index[0],sampled_data["visitor", "visits", "plant"].edge_label_index[1]):
                flag=True
                for ind,uv in enumerate(zip(sampled_data["visitor", "visits", "plant"].edge_index[0],sampled_data["visitor", "visits", "plant"].edge_index[1])):
                    u1,v1=uv
                    if(u1==u and v1==v):
                        temp.append(ground_truth_2[ind].tolist())
                        flag=False
                        break
                if(flag):
                    temp.append([0.0,0.0,0.0])

        ground_truth_2=torch.Tensor(temp)
        L2 = F.mse_loss(pred2, ground_truth_2)
        MSE+=L2
pred = preds[0]
ground_truth = ground_truths[0]
# !pip install torcheval
# print(ground_truth,pred)
# from torcheval.metrics import MulticlassAccuracy
# auc=MulticlassAccuracy()
# auc.update(torch.Tensor(ground_truth), torch.Tensor(pred))
# print(auc.compute())
# auc = roc_auc_score(ground_truth, pred)
print()
print("MSE : ",MSE)
print(f"Validation AUC: {auc:.4f}")
# print(preds[0].shape)